<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Alan Contreras

### **Link de repositorio de GitHub:** `https://github.com/AlanMontanares/MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/df/79/c80174d711ee26ee5da55a9cc3e248f1ec7a0188b5e4d6bbbbcd09b974b0/plotly-5.17.0-py2.py3-none-any.whl.metadata
  Using cached plotly-5.17.0-py2.py3-none-any.whl.metadata (7.0 kB)
Using cached plotly-5.17.0-py2.py3-none-any.whl (15.6 MB)
  Attempting uninstall: plotly
    Found existing installation: plotly 5.16.1
    Uninstalling plotly-5.16.1:
      Successfully uninstalled plotly-5.16.1



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [277]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [278]:
def custom_features(dataframe_in):

    df_copia = dataframe_in.copy()
    df_copia = df_copia.sort_values(["Customer ID", "InvoiceDate"])

    #################################### LENGHT

    # Calculamos la diferencia de dias entre la primera y ultima compra
    df_features = df_copia.groupby("Customer ID", observed=False)["InvoiceDate"].agg(lambda x: (x.iloc[-1] - x.iloc[0]).days).reset_index()
    df_features.columns = ["Customer ID", "Lenght"]

    #################################### RECENCY

    ultima_compra = df_copia.groupby("Customer ID", observed=False)["InvoiceDate"].last().reset_index()
    ultima_compra.columns = ["Customer ID", "Fecha última compra"]
    fecha_actual = pd.to_datetime("now")

    df_features["Recency"] = (fecha_actual - ultima_compra["Fecha última compra"]).dt.days

    ################################### MONETARY

    df_copia["Total gastado"] = df_copia["Price"] * df_copia["Quantity"]

    promedio_gastado = df_copia.groupby("Customer ID", observed=False)["Total gastado"].mean().reset_index()
    df_features["Monetary"] = promedio_gastado["Total gastado"]

    ################################### Frequency

    visitas_unicas = df_copia.groupby("Customer ID", observed=False)["Invoice"].nunique().reset_index()
    df_features["Frequency"] = visitas_unicas["Invoice"]

    ################################### PERIODICITY
    
    df_copia = df_copia.sort_values(["Customer ID", "InvoiceDate"])
    df_copia["IVT"] = df_copia.groupby("Customer ID", observed=False)["InvoiceDate"].diff().dt.days

    periocidad = df_copia.groupby("Customer ID", observed=False)["IVT"].std().reset_index()
    periocidad["IVT"].fillna(0, inplace=True) # Rellenamos con 0 a los que solo han ido 1 vez (ya que la diferncia cuando hay 1 valor arroja NaT)
    df_features["Periodicity"] = periocidad["IVT"] 


    df_features.set_index("Customer ID", inplace=True)

    return df_features

In [279]:
df_features = custom_features(df_retail)
df_features

,Lenght,Recency,Monetary,Frequency,Periodicity
Customer ID,,,,,
12346.0,196,4861,11.298788,11,21.724076
12347.0,37,4699,18.638310,2,4.422346
12348.0,0,4770,11.108000,1,0.000000
12349.0,181,4739,26.187647,3,16.200990
12351.0,0,4707,14.330000,1,0.000000
...,...,...,...,...,...
18283.0,275,4714,2.854240,6,11.783701
18284.0,0,4763,16.488571,1,0.000000
18285.0,0,4992,35.583333,1,0.000000


**Comentarios adicionales:** Para calcular la variable Monetary se considero que para una factura especifica el valor Price corresponde al valor unitario del producto en cuestion, por lo que el valor final que se pago fue el del precio multiplicado por la cantidad de unidades adquiridas. Por otro lado la variable Frenquency fue calculada contando los numeros de factura unicos para cada cliente, ya que una factura puede contener muchos pedidos los cuales fueron pagados en 1 misma visita. Por ultimo, todas las diferencias fueron calculadas en funcion de los dias.

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [280]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.min_values = X.min()
        self.max_values = X.max()
        return self

    def transform(self, X):
        X_transformed = (X - self.min_values) / (self.max_values - self.min_values)
        return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [304]:
#Se puede usar asi
pipeline = Pipeline([
    ('feature_extraction', FunctionTransformer(func=custom_features, validate=False)),  
    ('min_max_scaling', MinMax()),  
    ('tsne', TSNE(n_components=2, random_state=42))  
])

In [294]:
# O asi con ColumnTransformer() como pedia el enunciado

pipeline2 = Pipeline([
    ('feature_extraction', FunctionTransformer(func=custom_features, validate=False)),  
    ('min_max_scaling', ColumnTransformer(
            transformers=[
                ('min_max', MinMax(),['Lenght', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])  
            ],
            remainder='passthrough' 
        )),
    ('tsne', TSNE(n_components=2, random_state=42))  
])

In [302]:
tsne_result = pipeline2.fit_transform(df_retail)
tsne_df = pd.DataFrame(data=tsne_result, columns=['Dimension 1', 'Dimension 2'])

fig = px.scatter(tsne_df, x='Dimension 1', y='Dimension 2', title='Gráfico t-SNE')
fig.update_layout(width=800, height=700)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [290]:
k_values = range(1, 21)
inertia_values = []

for k in k_values:
    pipeline = Pipeline([
        ('feature_extraction', FunctionTransformer(func=custom_features, validate=False)),  
        ('min_max_scaling', ColumnTransformer(
                transformers=[
                    ('min_max', MinMax(),['Lenght', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])  
                ],
                remainder='passthrough' 
            )),
        ('kmeans', KMeans(n_clusters=k, random_state=42, n_init="auto"))
        ])
    pipeline.fit(df_retail)
    inertia_values.append(pipeline.named_steps['kmeans'].inertia_)

data = {
    'k': k_values,
    'inercia': inertia_values
}

fig = px.line(data, x='k', y='inercia', title='Método del Codo para Determinar el Número Óptimo de Clusters')
fig.update_traces(mode="markers+lines")
fig.update_layout(xaxis_title='Número de Clusters (k)', yaxis_title='Inercia')

fig.show()

**Respuesta:** En este caso los mejores valores de cluster estarian entre 3 y 6, ya que como el metodo dice, se busca el codo del grafico en donde la diminucion de inercia sea menos pronunciada, es decir, el punto donde el grafico comienza a aplanarse.

En este caso si se escoge un valor mayor a 6 traeria problemas, ya que se tendrian mas cluster, provocando mayor granularidad, esto traeria una perdida de comprension e interpretacion, ya que se perderian estructuras mas generales debido a este aumento de particiones. Tambien aumentaria la complejidad y tambien un riesgo de sobreajuste.

Dicho esto se tomara $k = 5$.



#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [311]:
pipeline = Pipeline([
        ('feature_extraction', FunctionTransformer(func=custom_features, validate=False)),  
        ('min_max_scaling', ColumnTransformer(
                transformers=[
                    ('min_max', MinMax(),['Lenght', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])  
                ],
                remainder='passthrough' 
            )),
        ('kmeans', KMeans(n_clusters=5, random_state=42, n_init="auto"))
        ])

pipeline.fit(df_retail)

df_features = custom_features(df_retail)

df_features['cluster'] = pipeline.predict(df_retail)

cluster_summary = df_features.groupby('cluster').mean()  

print(cluster_summary)

             Lenght      Recency   Monetary  Frequency  Periodicity
cluster                                                            
0         17.884120  4739.363090  31.904690   1.751073     2.469232
1        188.873757  4757.439364  27.265196   4.324056    21.033785
2         35.365260  4882.698052  37.908057   1.839286     5.314686
3          5.673118  5001.393548  71.723546   1.275269     1.165111
4        322.539548  4718.973635  36.027408  10.451036    20.042334


**Respuesta:** Se pueden ver cluster en los que hay clientes que no son fieles (cluster 3) y otros que son fieles (0,2) y otros con clientes muy fieles (1,4). Tambien se puede apreciar que los clientes mas fieles han ido a visitar (en promedio) mucho más la tienda, sin embargo su periocidad es alta en comparacion a los que han ido menos, aunque esto se podria inferir debido a que podria ocurrir que las pocas veces que estos han ido han sido en fechas muy proximas, por lo que la diferencia es pequeña y su desviacion estandar tambien.

Respecto a la variable de Rencency se puede ver que los clientes menos fieles son los que no visitan la tienda por mayores dias en comparacion al resto. Por ultimo respecto a la variable Monetary, se tiene que estos clientes menos fieles han gastado mucho mas dinero en promedio que el resto (quizas solo fueron a hacer una compra millonaria? quien sabe).

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [319]:
tsne_df["cluster"] = df_features['cluster'].values

fig = px.scatter(tsne_df, x='Dimension 1', y='Dimension 2', color='cluster', title='Gráfico t-SNE')
fig.update_layout(width=800, height=700)
fig.show()

**Respuesta:** Se separan muy bien los cluster en la proyeccion. Quizas probar con $k=6$ podria distinguir a grupos distintos que no aparecen con $k=5$, aunque esto tambien cambiaria probablemente los ya existentes, ya que por ejemplo el cluster numero 2 tiene una tendencia a ser tambien del cluster numero 1 y parte del cluster 3, entonces se podria formar un nuevo cluster en esta zona haciendo por ende que los promedios y distribuciones de los clusters existentes cambien. De igual forma con reducir los cluster.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>